In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv("training_dataset.csv")

In [3]:
data.head()

,Variable1,Variable2,Variable3,Variable4,Variable5,Variable6,Variable7,Variable8,Variable9,Variable10,...,Variable20,Variable21,Variable22,Variable23,Variable24,Variable25,Variable26,Variable27,Variable28,Target
0,1,ARO-4742,1,APHRNG1692,May-16,N,9.314022e+09,Col-6930,PKSA016674,USG1,...,NaN,NaN,VAR SULP TT3 PS,NaN,N,NaN,COM2,1150.0,MFTRC322,0
1,2,ARO-4928,2,APFPTV9946,Feb-17,N,9.251002e+09,Col-6930,PA2Z013889,USG4,...,NaN,NaN,VAR SULP TT4,NaN,N,NaN,COM1,1150.0,MFTRC322,0
2,3,ARO-3778,2,APHRNG1692,Dec-15,NaN,9.451552e+09,Col-4259,PCEM018562,USG4,...,NaN,NaN,VAR SULP DI TT3,NaN,N,NaN,COM3,1150.0,MFTRC322,0
3,4,ARO-4998,1,APFPTV154265,Sep-16,N,9.416109e+09,Col-4259,PTRW14933,USG4,...,NaN,NaN,VAR SULP DI TT3 PS,NaN,N,NaN,COM1,1000.0,MFTRC322,0
4,5,ARO-3627,2,APHRNG1692,Feb-16,N,9.500040e+09,Col-4259,PAM0026696,USG1,...,NaN,NaN,VAR SULP TT4 PS,NaN,N,NaN,COM1,1150.0,MAHEC465,0


In [4]:
data.shape

(199609, 29)

In [5]:
data.isnull().sum()

Variable1          0
Variable2          0
Variable3          0
Variable4          0
Variable5          0
Variable6      46391
Variable7      11535
Variable8          0
Variable9          0
Variable10         0
Variable11         0
Variable12     36818
Variable13         0
Variable14         0
Variable15         0
Variable16    174704
Variable17         0
Variable18     58539
Variable19         0
Variable20    175780
Variable21    199609
Variable22         0
Variable23    160126
Variable24         0
Variable25    182801
Variable26         0
Variable27     34163
Variable28         0
Target             0
dtype: int64

In [6]:
data1 = data.drop(['Variable16','Variable20','Variable21','Variable23', 'Variable25'], 1)

In [7]:
#since most of the values are NO ,lets impute the missing value with N
data1['Variable6'].fillna('N',inplace=True)

In [8]:
#not a normal distribution.lets impute missing value with median
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values=np.nan, strategy='median', axis=0)
data1[['Variable7']] = imputer.fit_transform(data1[['Variable7']])

C:\Users\sreekar chidurala\Anaconda3\envs\sreekar\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [9]:
data1['Variable12'].fillna(data1['Variable12'].value_counts().index[0],inplace=True)

In [10]:
#imputing null with 'N'   ( mode value)
data1['Variable18'].fillna('N',inplace=True)

In [11]:
data1[['Variable27']] = imputer.fit_transform(data1[['Variable27']])

In [12]:
data1.isnull().sum()

Variable1     0
Variable2     0
Variable3     0
Variable4     0
Variable5     0
Variable6     0
Variable7     0
Variable8     0
Variable9     0
Variable10    0
Variable11    0
Variable12    0
Variable13    0
Variable14    0
Variable15    0
Variable17    0
Variable18    0
Variable19    0
Variable22    0
Variable24    0
Variable26    0
Variable27    0
Variable28    0
Target        0
dtype: int64

In [13]:
from sklearn.preprocessing import OneHotEncoder

In [14]:
#80:20 split
from sklearn.model_selection import train_test_split

train, test = train_test_split(data1, test_size=0.2, random_state=0)

In [15]:
train.shape

(159687, 24)

In [16]:
test.shape

(39922, 24)

In [17]:
y_train=train["Target"]

In [18]:
y_test=test["Target"]

In [19]:
train=train.drop(labels="Target",axis=1)

In [20]:
test=test.drop(labels="Target",axis=1)

In [21]:
train.shape

(159687, 23)

In [22]:
test.shape

(39922, 23)

In [23]:
categorical = list(train.select_dtypes(include=['object']).columns.values)
for cat in categorical:
    train[cat]=train[cat].astype('str')

In [36]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(train)
train_enc=enc.transform(train)
test_enc=enc.transform(test)

In [37]:
train_enc ,test_enc

(<159687x173120 sparse matrix of type '<class 'numpy.float64'>'
 	with 3672801 stored elements in Compressed Sparse Row format>,
 <39922x173120 sparse matrix of type '<class 'numpy.float64'>'
 	with 877353 stored elements in Compressed Sparse Row format>)

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, make_scorer
SEED = 1
model_full_rf = RandomForestClassifier(n_estimators=5, class_weight='balanced', random_state=SEED, n_jobs=-1)
model_full_rf.fit(train_enc, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=5, n_jobs=-1, oob_score=False, random_state=1,
            verbose=0, warm_start=False)

In [27]:
model_full_rf.score(test_enc, y_test)

0.8906116928009619

In [38]:
#logestic
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
#from sklearn.feature_selection import RFE
#rfe = RFE(logreg, 13)             # running RFE with 13 variables as output
logreg.fit(train_enc,y_train)
  

C:\Users\sreekar chidurala\Anaconda3\envs\sreekar\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [39]:
logreg.score(test_enc, y_test)

0.9027854315916036